<a href="https://colab.research.google.com/github/VickkiMars/Transact/blob/master/cAlvin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import torch
from transformers import pipeline

In [17]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation",
                model="HuggingFaceH4/zephyr-7b-beta",
                torch_dtype=torch.bfloat16,
                device_map="auto"
                )

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
def format_prompt(query="", messages=False):
  if not messages:
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": query,
        }
    ]
  prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
  return prompt

In [52]:
user_input = input()

send ten thousand naira to etimbuk apkan 2134889302


In [9]:
def load_data():
  with open("data.json", 'r') as f:
    data = json.load(f)
  return data

In [13]:
def process_input(input):
  return load_data()[-1]['content'].format(input)

In [55]:
few_shot_prompt = format_prompt(messages=shots)

ValueError: Cannot use apply_chat_template() because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [15]:
outputs = pipe(few_shot_prompt, max_new_tokens=64, do_sample=True, top_p=0.95, temperature=0.01, num_return_sequences=1, return_full_text=False)

KeyboardInterrupt: 

In [43]:
res = outputs[0]["generated_text"]

100000 | West Bank | 2196853023 | Maxwell McDavies


In [1]:
# amount | bank name | account number | name
response = res.split('|')
def process_text(response):
  rs_js = {}
  if len(response) == 4:
    rs_js['amount'] = int(response[0].strip())
    rs_js['bank_name'] = response[1].lower().strip()
    rs_js['account_number'] = response[2].strip()
    rs_js['name'] = response[3].lower().strip()

    assert isinstance(rs_js['amount']) == int
    assert isinstance(rs_js['bank_name']) == str
    assert isinstance(rs_js['account_number']) == int
    assert len(str(rs_js['account_number'])) == 10
    assert isinstance(rs_js['name']) == str

  elif len(response) == 3 and isinstance(response[1]) == str :
    rs_js['amount'] = response[0].strip()
    rs_js['bank_name'] = response[1].lower().strip()
    rs_js['account_number'] = response[2].lower().strip()
  else:
    rs_js['amount'] = response[0].strip()
    rs_js['account_number'] = response[1].lower().strip()

  return rs_js
